<a href="https://colab.research.google.com/github/AhanR/Maximl-internship-task/blob/main/tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Problem statement

In [ ]:
words = ["abcd", "dbca", "xyz", "def", "zyx", "cadb", "aabcd","ghi","cabda","abaa","abba"]

def execute_part1(words):
  res = {}
  # create a hash and add it to the dict with the other words with the same hash or create new entry if the hash is new
  for word in words:
    hash = wordHash(word)
    if hash in res:
      res[hash].append(word)
    else:
      res[hash] = [word]
  for key in res:
    print(" ".join(res[key]))

# create a character index based hashing technique
def wordHash(word):
  # the 0 is the frequency of the corresponding character appearing in the word
  # every index can be converted to a letter starting at index 0-> character 'a'
  hash = [0]*26
  for c in word:
    hash[ord(c)-ord("a")] += 1
  return str(hash)

In [ ]:
execute_part1(words)

abcd dbca cadb
xyz zyx
def
aabcd cabda
ghi
abaa
abba


# Second Problem Statement

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [152]:
def execute_part2(inputFile, outputFile = "/content/part2_output.mp4"):
  # VIDEO_FILE -> the input video file
  # VIDEO_OUT -> the file to be written to
  VIDEO_FILE = inputFile
  VIDEO_OUT = outputFile

  # the hard upper bound below which the video does not need to be edited
  UPPER_BOUND = 25

  # cap -> video stream object
  cap = cv2.VideoCapture(VIDEO_FILE)

  # all the video properties
  # n_frames -> number of frames in the video
  # in_fps -> frames per second of the video
  n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  in_fps = cap.get(cv2.CAP_PROP_FPS)
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  # out_fps -> frames per second to be set in the output video (20 by default) or lower if the source video has lower than 25 fps
  # codec -> the codec used to encode the video, other available options are MP4V
  # fourcc -> fourcc codec code
  # out -> the output stream
  out_fps = 20 if in_fps > UPPER_BOUND else in_fps
  codec = 'DIVX'
  fourcc = cv2.VideoWriter_fourcc(*codec)
  out =  cv2.VideoWriter(VIDEO_OUT, fourcc, out_fps, (width, height))

  print("INPUT VIDEO STATS:")
  print(f"frames : {n_frames} fps : {in_fps:0.2f} len : {n_frames/in_fps}")
  print(f"w : {width} h : {height}")
  print()
  print("OUTPUT SETTINGS:")
  print(f"fps : {out_fps} codec : {codec} filePath : {VIDEO_OUT}")
  print()

  # the time is in milli seconds
  # tx -> the time taken per frame in the input video (source video)
  # ty -> the time taken per fram in the output video (target video)
  tx = 1000/in_fps 
  ty = 1000/out_fps

  # outFrameNumber -> tracks the number of frames written to the target
  # directOutFlag -> sets to true when we need to pass the video as is, i.e. when in_fps < 25
  outFrameNumber = 0
  directOutFlag = False if in_fps > UPPER_BOUND else True

  for frame in tqdm(range(n_frames)):
    # ret -> tells us if the next frame was read or not (True if successful and False if no more frames or not successful)
    # img -> the next frame as an image
    ret, img = cap.read()
    if ret == False:
      print("Operation Aborted")
      break
    # converting from BGR to GRAYSCALE and back to get the greyscale image in BGR colour space
    # adding in text
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img = cv2.putText(img, "Open CV", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0))

    # if direct out is set, push data to out stream or else check if current frame must be sampled
    if directOutFlag or outFrameNumber*ty <= frame*tx:
      # write to output
      # increment the total number of frames sampled (outFrameNumber)
      out.write(img)
      outFrameNumber+=1

  out.release()
  cap.release()
  print(f"\nWritten frames {outFrameNumber}")

  print()

  # some video stats for the new video file to cross check some values
  cap = cv2.VideoCapture(VIDEO_OUT)
  final_fps = cap.get(cv2.CAP_PROP_FPS)
  n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print("OUTPUT VIDEO STATS:")
  print(f"frames : {n_frames} fps : {final_fps} len : {n_frames/final_fps}")

In [153]:
execute_part2("/content/input4.mp4")

INPUT VIDEO STATS:
frames : 4053 fps : 30.00 len : 135.1
w : 1280 h : 542

OUTPUT SETTINGS:
fps : 20 codec : DIVX filePath : /content/part2_output.mp4



100%|██████████| 4053/4053 [00:37<00:00, 109.48it/s]


Written frames 2702

OUTPUT VIDEO STATS:
frames : 2702 fps : 20.0 len : 135.1
